In [1]:

from google_play_scraper import reviews, Sort
import pandas as pd
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 定义智能抓取函数
# 创建函数，参数：package_name: 要抓取的应用包名.
# target_per_rating: 每个评分等级的目标抓取数量（默认5000条）
# total_desired: 期望的总评论数（默认20000条）
def smart_review_scraper(package_name, target_per_rating=5000, total_desired=20000):
    all_reviews = []
    error_count = 0
    max_errors = 5

    # 优先尝试抓取低分评论，从最有帮助的排序开始，这通常包含更多历史数据
    for score in [1, 2, 3, 4, 5]:
        print(f"正在抓取 {score} 星评论...")
        token = None
        score_reviews = []
        
        with tqdm(total=target_per_rating, desc=f'{score}星评论') as pbar:
            while len(score_reviews) < target_per_rating and error_count < max_errors:
                try:
                    # 对低分评论使用 MOST_RELEVANT 排序，这有助于获取经典的低分评价
                    sort_method = Sort.MOST_RELEVANT if score <= 3 else Sort.NEWEST
                    
                    batch, token = reviews(
                        package_name,
                        lang='en',
                        country='us',
                        sort=sort_method,
                        count=100,
                        continuation_token=token
                    )
                    
                    # 按当前目标评分筛选
                    if batch and 'score' in batch[0]:
                        batch_filtered = [r for r in batch if r.get('score') == score]
                        score_reviews.extend(batch_filtered)
                        pbar.update(len(batch_filtered))
                    else:
                        # 如果返回的数据中没有score字段，则全部加入
                        score_reviews.extend(batch)
                        pbar.update(len(batch))

                    if token is None or not batch:
                        print(f"  没有更多 {score} 星评论了。")
                        break

                    time.sleep(1)  # 礼貌性等待

                except Exception as e:
                    error_count += 1
                    print(f"获取 {score} 星评论时出错 (错误 #{error_count}): {e}")
                    time.sleep(5)
                    
                    if error_count >= max_errors:
                        print("错误次数过多，停止抓取。")
                        break

        all_reviews.extend(score_reviews)
        print(f"已抓取 {len(score_reviews)} 条 {score} 星评论。")

        # 如果总数据量已满足要求，可提前结束
        if len(all_reviews) >= total_desired:
            print(f"总评论数已达目标，停止抓取。")
            break

    return pd.DataFrame(all_reviews)

In [3]:
# 设置应用包名并执行抓取
# ChatGPT在Google Play商店的唯一标识符。
app_package = "com.openai.chatgpt"

# 执行抓取 - 目标是每个评分抓取约4000条，总共约20000条
print("开始抓取ChatGPT应用评论...")
reviews_df = smart_review_scraper(app_package, target_per_rating=4000, total_desired=20000)
print("抓取完成！")

开始抓取ChatGPT应用评论...
正在抓取 1 星评论...


1星评论: 4004it [04:30, 14.80it/s]                                              


已抓取 4004 条 1 星评论。
正在抓取 2 星评论...


2星评论: 4009it [34:38,  1.93it/s]                                              


已抓取 4009 条 2 星评论。
正在抓取 3 星评论...


...

已抓取 4001 条 3 星评论。
正在抓取 4 星评论...


4星评论: 4008it [11:01,  6.06it/s]                                              


已抓取 4008 条 4 星评论。
正在抓取 5 星评论...


5星评论: 4011it [01:20, 49.98it/s]                                              

已抓取 4011 条 5 星评论。
总评论数已达目标，停止抓取。
抓取完成！


In [6]:
# 检查并保存数据
if not reviews_df.empty:
    # 确保数据包含版本信息
    if 'reviewCreatedVersion' not in reviews_df.columns:
        print("警告: 部分评论可能缺少 'reviewCreatedVersion' 字段。")
    
    # 显示基本统计信息
    print(f"\n成功抓取 {len(reviews_df)} 条评论。")
    print("\n评分分布:")
    print(reviews_df['score'].value_counts().sort_index())
    
    # 检查时间范围
    reviews_df['at'] = pd.to_datetime(reviews_df['at'])
    print(f"\n数据时间范围: {reviews_df['at'].min()} 到 {reviews_df['at'].max()}")
    
    # 保存到CSV文件
    csv_filename = 'chatgpt_reviews_enhanced.csv'
    reviews_df.to_csv(csv_filename, index=False)
    print(f"\n数据已保存到 '{csv_filename}'")
    
  
else:
    print("未获取到任何数据。")


成功抓取 20033 条评论。

评分分布:
score
1    4004
2    4009
3    4001
4    4008
5    4011
Name: count, dtype: int64

数据时间范围: 2023-07-25 08:31:59 到 2025-10-16 13:19:26

数据已保存到 'chatgpt_reviews_enhanced.csv'
